cursor.execute("alter table rides modify column status varchar(256) primary key;")
conn.commit()

In [4]:
# Markdown: rides.ipynb
# Purpose: create ride requests, accept/reject, pickup/dropoff flows and compute wait/complete times.

import mysql.connector
import pandas as pd
import bcrypt
from datetime import datetime, timedelta
from geopy.distance import geodesic

DB_CONFIG = {"host":"localhost","user":"root","password":"40418","database":"aeras"}
def get_conn(): return mysql.connector.connect(**DB_CONFIG)
def now_bd(): return datetime.utcnow() + timedelta(hours=6)

# ID helper for rides (UUID-like or custom)
import uuid
def gen_rideid():
    return "RIDE-" + uuid.uuid4().hex[:12]


In [5]:
# Create a ride request
def create_ride(userid, start, dest):
    rideid = gen_rideid()
    now = now_bd()
    conn = get_conn()
    with conn.cursor() as cur:
        cur.execute("""
            INSERT INTO rides (rideid, request_time, userid, start, dest, status, created_at)
            VALUES (%s,%s,%s,%s,%s,%s,%s)
        """, (rideid, now, userid, start, dest, "REQUESTED", now))
        conn.commit()
    conn.close()
    return rideid

# Puller accepts ride (race-safe: check current status)
def accept_ride(rideid, pullerid):
    conn = get_conn()
    now = now_bd()
    with conn.cursor() as cur:
        # check current status
        cur.execute("SELECT status, request_time FROM rides WHERE rideid=%s FOR UPDATE", (rideid,))
        row = cur.fetchone()
        if not row:
            conn.close(); return {"error":"ride not found"}
        if row[0] != "REQUESTED":
            conn.close(); return {"error":"ride not available", "current_status": row[0]}
        # accept
        cur.execute("UPDATE rides SET status=%s, accepted_time=%s, pullerid=%s WHERE rideid=%s", ("ACCEPTED", now, pullerid, rideid))
        # compute wait_seconds
        request_time = row[1]
        if request_time:
            wait_seconds = int((now - request_time).total_seconds())
            cur.execute("UPDATE rides SET wait_seconds=%s WHERE rideid=%s", (wait_seconds, rideid))
            # update puller avg wait: simple incremental avg
            cur.execute("SELECT avg_wait_time_seconds, ride_count FROM pullers WHERE pullerid=%s", (pullerid,))
            r = cur.fetchone()
            if r and r[0] is not None:
                prev_avg = r[0]; prev_count = r[1] or 0
                new_avg = (prev_avg * prev_count + wait_seconds) / (prev_count + 1)
            else:
                new_avg = wait_seconds
                prev_count = r[1] if r else 0
            cur.execute("UPDATE pullers SET avg_wait_time_seconds=%s WHERE pullerid=%s", (new_avg, pullerid))
        conn.commit()
    conn.close()
    return {"status":"accepted", "rideid": rideid}


In [7]:
# Puller confirms pickup
def confirm_pickup(rideid, pullerid):
    now = now_bd()
    conn = get_conn()
    with conn.cursor() as cur:
        cur.execute("UPDATE rides SET status=%s, pickup_time=%s WHERE rideid=%s AND pullerid=%s", ("ACTIVE", now, rideid, pullerid))
        conn.commit()
    conn.close()
    return True
# Dropoff & awarding (distance measured by GPS on rickshaw side)
def confirm_dropoff(rideid, pullerid, current_lat, current_long, block_lat, block_long):
    now = now_bd()
    dist_m = geodesic((current_lat, current_long), (block_lat, block_long)).meters
    conn = get_conn()
    with conn.cursor() as cur:
        # update distances and drop time
        cur.execute("UPDATE rides SET drop_time=%s, distance_to_block_m=%s WHERE rideid=%s AND pullerid=%s", (now, dist_m, rideid, pullerid))
        # compute completion time if pickup_time exists
        cur.execute("SELECT pickup_time FROM rides WHERE rideid=%s", (rideid,))
        pickup_time = cur.fetchone()[0]
        if pickup_time:
            pickup_dt = pickup_time
            completion_seconds = int((now - pickup_dt).total_seconds())
            cur.execute("UPDATE rides SET completion_seconds=%s, completion_time=%s WHERE rideid=%s", (completion_seconds, now, rideid))
            # update puller avg completion time (incremental)
            cur.execute("SELECT avg_completion_time_seconds, ride_count FROM pullers WHERE pullerid=%s", (pullerid,))
            r = cur.fetchone()
            if r and r[0] is not None:
                prev_avg = r[0]; prev_count = r[1] or 0
                new_avg = (prev_avg * prev_count + completion_seconds) / (prev_count + 1)
            else:
                new_avg = completion_seconds
            cur.execute("UPDATE pullers SET avg_completion_time_seconds=%s WHERE pullerid=%s", (new_avg, pullerid))
        # award points using points module function (import style)
        conn.commit()
    conn.close()

    #award points (use function from points&rewards notebook)
    #Example: award_points_for_drop(pullerid, dist_m, rideid=rideid)
    return {"distance_m": dist_m}
    # Helper to get current ride state
def get_ride(rideid):
    conn = get_conn()
    df = pd.read_sql("SELECT * FROM rides WHERE rideid=%s", conn, params=(rideid,))
    conn.close()
    return df



SyntaxError: invalid syntax (6170533.py, line 38)